In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings
sys.path.append("../src/") # go to parent dir
from main import get_X, get_y, get_score, get_train_test
from models import get_model_default, show_importance
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection
import xgboost as xgb
import matplotlib.pyplot as plt

## Read Train data

In [ ]:
train_data = get_X('train')
train_scores = get_y()

## Benchmark

We try to model if AWAY wins

### train, valid test split

In [ ]:
X_train, y_train, X_test, y_test, X_valid, y_valid = get_train_test(train_size=0.8, random_state=42)

### xgb model

In [ ]:
model = get_model_default(X_train, y_train, X_valid, y_valid)

### Some analysis

In [ ]:
show_importance(model)

### Accuracy on test set

A simple benchmark is always predicting home wins

In [ ]:
target = train_scores.loc[X_test.index].copy()

home_wins = target
home_wins = 0 * home_wins
home_wins.iloc[:,0] = 1

np.round(accuracy_score(home_wins,target),4)

Using our model

In [ ]:
X_test_xgb = xgb.DMatrix(X_test)

predictions = bst.predict(X_test_xgb, iteration_range=(0, bst.best_iteration))
predictions = pd.DataFrame(predictions)

get_score(predictions, target)

## Submission

In [ ]:
test_data = get_X('test')

In [ ]:
X_bench = xgb.DMatrix(test_data)

predictions = bst.predict(X_bench, iteration_range=(0, bst.best_iteration))
predictions = pd.DataFrame(predictions)

predictions[2] = 0
predictions.columns = [0,2,1]
predictions = (predictions.reindex(columns=[0,1,2]).rank(1,ascending=False)==1).astype(int)

predictions.columns = ['HOME_WINS', 'DRAW', 'AWAY_WINS']
predictions.index = test_data.index
submission = predictions.reset_index()
submission.to_csv('./benchmark_submission.csv', index=False)
submission.head()